In [1]:
# Dependencies and Credentials
import requests
import tweepy
import pandas as pd
from config import api_key

In [11]:
# Define Permissions
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAALe%2BvgEAAAAAcLsxdbFdxXNYG7lC9Fs6fPBwvSM%3DLK5VO6DZjTnshVMxsgva9djxz6DOu6C9mi97oGIV1Pl9dY7KGT"

In [13]:
# Define URL
base_url = 'https://api.x.com/2/tweets/search/recent?'
# base_url = 'https://api.twitter.com/2/tweets/search/recent?'

In [14]:
# Define parameters
query_params = {
    'query': 'US presidential election',  # Your search query
    'tweet.fields': 'author_id,context_annotations,created_at',  # Fields to return
    'max_results': 100,  # Number of results per request (max 100)
    'start_time': '2016-01-01T00:00:00Z',  # Start time for the search
    'end_time': '2016-12-31T23:59:59Z'  # End time for the search
}

In [15]:
# Prepare headers
headers = {
    'Authorization': f'Bearer {BEARER_TOKEN}'
}

In [16]:
# Function to fetch tweets
def fetch_tweets(base_url, query_params, headers):
    response = requests.get(base_url, params=query_params, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return JSON data
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.json())  # Print the error response
        return None

In [17]:
# Fetch tweets
data = fetch_tweets(base_url, query_params, headers)

Request failed with status code 403
{'client_id': '29277879', 'detail': 'When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.', 'registration_url': 'https://developer.twitter.com/en/docs/projects/overview', 'title': 'Client Forbidden', 'required_enrollment': 'Appropriate Level of API Access', 'reason': 'client-not-enrolled', 'type': 'https://api.twitter.com/2/problems/client-forbidden'}


In [ ]:
# Process and save the results
if data:
    tweets = data.get('data', [])
    # Create a DataFrame
    df = pd.DataFrame([
        {
            'id': tweet.get('id'),
            'author_id': tweet.get('author_id'),
            'created_at': tweet.get('created_at'),
            'context_annotations': tweet.get('context_annotations')
        }
        for tweet in tweets
    ])
    # Save to CSV
    df.to_csv('tweets_2016_with_details.csv', index=False)

    print("Data saved to tweets_2016_with_details.csv")
else:
    print("No data to save.")

In [10]:
# # Authenticate to Twitter
# client = tweepy.Client(bearer_token=BEARER_TOKEN)

# client = tweepy.Client(
consumer_key="s2BlNvYCXNItf3ja8jAS6FVcA"
consumer_secret="e6UnSltTqNBXI3ZV1IU1iPyMd7iN4jdb8m1gH06YYsbzSSqWV7"
access_token="1830755630476578816-NjvKZXCfyqUnoaZ4lN6hjyNIA9qkDo"
access_token_secret="tdGuBaW8MGBINrhajSoOz3IoH1BKI3IjTSrMDm4EQoeFt"
# )


auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth)
public_tweets = api.search_tweets(q='election') #q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)
for tweet in public_tweets:
    print(tweet.text)

def fetch_tweets(query, start_time, end_time, max_results=100):
    tweets = []
    for tweet in tweepy.Paginator(client.search_recent_tweets,
                                  query=query,
                                  start_time=start_time,
                                  end_time=end_time,
                                  tweet_fields=['created_at', 'geo', 'place'],
                                  max_results=max_results).flatten(limit=1000):
        # Extract location information
        location = None
        if tweet.geo:
            location = tweet.geo['place_id']  # For geotagged tweets
        elif tweet.place:
            location = tweet.place['full_name']  # For tweets with place information

        # Handle cases where location data might be missing
        location_info = tweet.place['full_name'] if tweet.place else None

        tweets.append([tweet.id, tweet.text, tweet.created_at, location_info])
    return tweets

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [3]:
# Example usage for the year 2016
tweets_2016 = fetch_tweets("US presidential election", "2016-01-01T00:00:00Z", "2016-12-31T23:59:59Z")
df_2016 = pd.DataFrame(tweets_2016, columns=['id', 'text', 'created_at', 'location'])
df_2016.to_csv('tweets_2016_with_location.csv', index=False)

Unauthorized: 401 Unauthorized
Unauthorized

In [12]:
import requests
import os
import json

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get(BEARER_TOKEN)


def create_url():
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=1278747501642657792,1255542774432063488"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


# def main():
url = create_url()
json_response = connect_to_endpoint(url)
print(json.dumps(json_response, indent=4, sort_keys=True))


# if __name__ == "__main__":
#     main()

401


Exception: Request returned an error: 401 {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}